In [8]:
#aqui ya no se explica mucho todo lo realizado porque se trato igual la data como en el de arboles de decisión
#importamos las librerias necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

In [9]:
# Cargamos los datos
datos = pd.read_csv("train.csv")

In [10]:
#llenamos con el promedio las edades faltantes
#borramos columnas que no nos aportan nada al modelo
datos["Age"] = datos["Age"].fillna(datos["Age"].mean())
datos = datos.drop(["Cabin"], axis=1)
datos = datos.dropna()
datos = datos.drop(["Name", "PassengerId", "Ticket"], axis=1)


In [11]:
#pasamos a numericas las variables categóricas
dummies_sex = pd.get_dummies(datos["Sex"], drop_first=True)
datos = datos.join(dummies_sex)
datos = datos.drop(["Sex"], axis=1)
dummies_embarked = pd.get_dummies(datos["Embarked"], drop_first=True)
datos = datos.join(dummies_embarked)
datos = datos.drop(["Embarked"], axis=1)

### MODELO MANUAL BAYES

In [13]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# COMO LOS DATOS YA ESTAN TRAJADOS EMPEZAMOS DE ACA

# Dividir en entrenamiento y prueba
X = datos.drop("Survived", axis=1)
y = datos["Survived"]
X_ent, X_pru, y_ent, y_pru = train_test_split(X, y, test_size=0.2, random_state=42)

# Unir X e y de entrenamiento
train_df = X_ent.copy()
train_df["Survived"] = y_ent

# Calcular P(clase)
p_clase = train_df["Survived"].value_counts(normalize=True).to_dict()

# Calcular P(x_i | clase)
cond_probs = {}
for col in X.columns:
    cond_probs[col] = {}
    for cls in p_clase.keys():
        subset = train_df[train_df["Survived"] == cls]
        probs = subset[col].value_counts(normalize=True).to_dict()
        cond_probs[col][cls] = probs

# Función para predecir una fila
def predecir(fila):
    log_probs = {}
    for cls in p_clase:
        log_prob = np.log(p_clase[cls])
        for col in X.columns:
            valor = fila[col]
            prob = cond_probs[col].get(cls, {}).get(valor, 1e-6)
            log_prob += np.log(prob)
        log_probs[cls] = log_prob
    return max(log_probs, key=log_probs.get)

# Predecir todo el conjunto de prueba
y_pred = X_pru.apply(predecir, axis=1)

# Evaluación
print("Accuracy (Naive Bayes manual):", accuracy_score(y_pru, y_pred))


Accuracy (Naive Bayes manual): 0.7584269662921348
